<a href="https://colab.research.google.com/github/dmswl0707/Face-Expression-Recognition_with-DNN/blob/main/training_val.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import os
from PIL import Image
import random
import numpy as np
import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets

from torch.utils.data import DataLoader
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary

# ckplus_res 데이터 다운받고 불러오기
root = 'C:/Users/ChoiEunJi/Desktop/ckplus_res'

#불러온 데이터 로드하기

def load_data():
    transforms = transforms.Compose([transforms.ToTensor(), transforms.Nomalize((),())])

    train_dataset = torchvision.datasets.ckplus_res(root, train = True , download = True)
    test_dataset = torchvision.datasets.ckplus_res(root, test = True, download = True)

def split_train_val_test(train_dataset, test_dataset, p=0.8):
    train_set_num = int(len(train_dataset.data)*p)
    
    train_data, val_data = torch.utils.data.random_split(train_dataset, [train_set_num, len(train_dataset)-train_set_num])
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=32, shuffle=True)
    test_loader=torch.utils.data.DataLoader(test_dataset, batch_size, shuffle=True)
    
    return train_loader, val_loader, test_loader

#로드 할때 train 데이터에 20%는 validation을 위해 나



In [ ]:
num_classes = 7

num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')

labels[0:29]=0 #30
labels[30:59]=1 #29
labels[60:92]=2 #32
labels[93:124]=3 #31
labels[125:155]=4 #30
labels[156:187]=5 #31
labels[188:]=6 #30

names = ['ANGRY','DISGUST','FEAR','HAPPY','NEUTRAL','SAD','SURPRISE']

def getLabel(id):
    return ['ANGRY','DISGUST','FEAR','HAPPY','NEUTRAL','SAD','SURPRISE'][id]

In [ ]:
# convert class labels to on-hot encoding# conve 
Y = np_utils.to_categorical(labels, num_classes)

#Shuffle the dataset
x,y = shuffle(img_data,Y, random_state=2)
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=2)

In [ ]:
def conv_block_1(in_dim, out_dim, act_fn, stride=1):
    model=nn.Sequential(
        nn.Conv2d(in_dim,out_dim, kernerl_size=1, stride=stride),
        act_fn, ##???
    )
    return model

def conv_block_3(in_dim, out_dim, act_fn):
    model=nn.Sequential(
        nn.Conv2d(in_dim, out_dim, kernel_size=3, stride=1, padding=1),
        act_fn,
    )
    return model 

class BottleNeck(nn.Module):
    def __init__(self, in_dim, mid_dim, out_dim, act_fn,down=False):
        super(BottleNeck, self).__init__()
        self.act_fn=act_fn
        self.down=down

        if self.down:
            self.layer=nn.Sequential(
                conv_block_1(in_dim, mid_dim, act_fn,2),
                conv_block_3(mid_dim, mid_dim, act_fn),
                conv_block_1(mid_dim,out_dim, act_fn),
            )
            self.downsample=nn.Conv2d(in_dim, out_dim,1,2)
        else:
            self.layer=nn.Sequential(
                conv_block_1(in_dim, mid_dim, act_fn),
                conv_block_3(mid_dim, mid_dim, act_fn),
                conv_block_1(mid_dim, out_dim, act_fn),
            )
            self.dim_equalizer=nn.Conv2d(in_dim, out_dim, kernel_size=1)

        def forward(self, x):
            if self.down:
                dpwnsample=self.downsample(x)
                out=self.layer(x)
                out=out+downsample
            else:
                out=self.layer(x)
                if x.size() is not out.size():
                    x=self.dim_equalizer(x)
                out=out+x
            return out

class ResNet(nn.Module):

    def __init__(self, base_dim, num_classes=2):
        super(ResNet, self).__init__()
        self.act_fn=nn.ReLU()
        self.layer_1=nn.Sequential(
            nn.Conv2d(3, base_dim,7,2,3),
            nn.ReLU(),
            nn.MaxPool2d(3,2,1),
        )
        self.layer_2=nn.Sequential(
            BottleNeck(base_dim, base_dim, base_dim*4, self.act_fn),
            BottleNeck(base_dim*4, base_dim, base_dim*4, self.act_fn),
            BottleNeck(base_dim*4, base_dim, base_dim*4, self.act_fn, down=True),
        )
        self.layer_3=nn.Sequential(
            BottleNeck(base_dim*4, base_dim*2, base_dim*8, self.act_fn),
            BottleNeck(base_dim*8, base_dim*2, base_dim*8, self.act_fn),
            BottleNeck(base_dim*8, base_dim*2, base_dim*8, self.act_fn),
            BottleNeck(base_dim*8, base_dim*2, base_dim*8, self.act_fn, down=True),
        )
        self.layer_4=nn.Sequential(
            BottleNeck(base_dim*8, base_dim*4, base_dim*16, self.act_fn),
            BottleNeck(base_dim*16, base_dim*4, base_dim*16, self.act_fn),
            BottleNeck(base_dim*16, base_dim*4, base_dim*16, self.act_fn),
            BottleNeck(base_dim*16, base_dim*4, base_dim*16, self.act_fn),
            BottleNeck(base_dim*16, base_dim*4, base_dim*16, self.act_fn),
            BottleNeck(base_dim*16, base_dim*4, base_dim*16, self.act_fn, down=True),
        
        self.layer_5=nn.Sequential(
            BottleNeck(base_dim*16, base_dim*8, base_dim*32, self.act_fn),
            BottleNeck(base_dim*32, base_dim*8, base_dim*32, self.act_fn),
            BottleNeck(base_dim*32, base_dim*8, base_dim*32, self.act_fn),
        )
        self.avgpool= nn.AvgPool2d(7,1)
        self.fc_layer=nn.Linear(base_dim*32, num_classes)

        def forward(self,x):
            out=self.layer_1(x)
            out=self.layer_2(out)
            out=self.layer_3(out)
            out=self.layer_4(out)
            out=self.layer_5(out)
            out=self.avgpool(out)
            out=out.view(batch_size, -1)
            out=self.fc_layer(out)
            return out

In [ ]:
#train 및 test하는 메소
def train(epoch, model, train_loader, val_loader):
    #model.train()

    train_loss = 0.0
    train_acc = 0.0
    global_step = 0
    for e in range(epoch):
        for batch_idx, (input, target) in enumerate(train_loader):
            global_step = e * len(train_loader) * batch_size + (batch_idx * batch_size)

            input, target = input.cuda(), target.cuda()
            input, target = Variable(input), Variable(target)

            optimizer.zero_grad()

            output = model(input)
            loss=F.nll_loss(output, target)
            preds = output.data.max(dim=1, keepdim=True)[1]
            acc=preds.eq(target.data.view_as(preds)).cpu().sum()
            train_loss +=F.nll_loss(output, target, reduction='sum').data.item()
            train_acc +=acc.item()

            loss.backward()
            optimizer.step()

        with torch.no_grad():
            val_acc, val_loss = test(model, val_loader)
            writer.add_scalar('Loss/val', val_loss, global_step)
            writer.add_scalar('Accuracy/val', val_acc, global_step)

        loss = train_loss/len(train_loader.dataset)
        accuracy = 100.0 * train_acc/len(train_loader.dataset)
        writer.add_scalar('Loss/train', loss, global_step)
        writer.add_scalar('Accuracy/train', accuracy, global_step)

        print('Epoch: {0:4} iter: {1:10} loss {2:3.5f} acc {3:3.5f} \t v_loss {4:3.5f} v_acc {5:3.5f}'
              .format(e+1, global_step, loss, accuracy, val_loss, val_acc))
        
        train_loss = 0.0
        train_acc = 0.0



In [ ]:
def test(model, data_loader):
    #model.eval()

    running_acc = 0.0
    running_loss = 0.0
    for batch_idx, (input, target) in enumerate(data_loader):

        input, target = input.cuda(), target.cuda()
        input, target = Variable(input), Variable(target)

        output = model(input)

        preds = output.data.max(dim=1, keepdim=True)[1]
        acc = preds.eq(target.data.view_as(preds)).cpu().sum()
        running_loss +=F.nll_loss(output, target, reduction='sum').data.item()
        running_acc += acc.item()

    accuracy = 100.0 * running_acc / len(data_loader.dataset)
    loss = running_loss / len(data_loader.dataset)

    return accuracy, loss



In [ ]:
if __name__=="__main__":
    writer = SummaryWriter()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    #hyperparameter
    lr = 0.003
    epoch = 50
    batch_size = 512
    train_val_p=0.8

    #데이터 셋 로드 및 train, val, test로 데이터 셋 나눔
    train_dataset, test_dataset = load_mnist()
    train_loader, val_loader, test_loader = split_train_val_test(train_dataset, test_dataset, p=train_val_p)

    #모델 불러오고, optimizer 설정
    model = Net().to(device)                            # 모델 불러오기
    optimizer = optim.SGD(model.parameters(), lr)      #optimizer 설정

    #layer 정보 프린트 하기
    print(summary(model, input_size=(1, 28, 28)))

    train(epoch, model, train_loader, val_loader)   #학습
    acc, _ = test(model, test_loader)               #테스트

    print('test acc {0:3.5f}' .format(acc))
    writer.close()